In [1]:
import sql_tools
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.finance import candlestick_ohlc

fire = sql_tools.IcePick(sqlite3, 'fire.db')

/home/ryan/.local/lib/python3.5/site-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
def chart_candles(candles):
        ohlc = []
        for index in candles.index:
            time = index
            open_= candles.loc[index, 'open']
            high = candles.loc[index, 'high']
            low= candles.loc[index, 'low']
            close = candles.loc[index, 'close']
            ohlc.append([time, open_, high, low, close])
        
        fig = plt.figure()
        fig.set_size_inches(13, 7)
        ax1 = plt.subplot2grid((1,1), (0,0))
        candlestick_ohlc(ax1, ohlc, width=0.4)
        plt.show()

In [3]:
pair_names = list(fire.read_sql("SELEcT * from sqlite_master where type = 'table'")['name'])

In [4]:
pair_data = {}
def get_data(currency):
    data = fire.read_sql("SELECT * FROM [{}]".format(currency))
    data.index = data['time']
    data.sort_values('time', inplace = True)
    return data

for pair_name in pair_names:
    pair_data[pair_name] = get_data(pair_name)

In [5]:
print(list(pair_data.keys()))

['BTC-EUR', 'LTC-EUR', 'LTC-BTC', 'BTC-GBP', 'BTC-USD', 'ETH-USD', 'LTC-USD', 'ETH-BTC', 'ETH-EUR']


In [6]:
def win_condition(candles_in, prediction_gap = 2):
    result = []
    for i, index in enumerate(candles.index):
        if i < prediction_gap:
            result.append(0)
        if i >= prediction_gap :
            current_low = candles.loc[index, 'low']
            last_high = candles.iloc[i - prediction_gap]['low']
            if current_low < last_high:
                result.append(1)
            if current_low >= last_high:
                result.append(0)
    return result




In [11]:
x = pair_data['BTC-USD']

In [12]:
time_diffs = []
last_index = x.index[0]
for index in x.index:
    time_diffs.append(index - last_index)
    last_index = index
    
time_diffs = np.array(time_diffs)
x['time_diffs'] = time_diffs

x.loc[:1513451400]